<a href="https://colab.research.google.com/github/subha9542/Datamining/blob/master/Data_mining_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from itertools import combinations 
eps = np.finfo(float).eps
from numpy import log2 as log
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [2]:
!pwd
df = pd.read_csv("HW5.csv")
df.head

/content


<bound method NDFrame.head of    department  status     age    salary  count
0       sales  senior  31..35  46k..50k     30
1       sales  junior  26..30  26k..30k     40
2       sales  junior  31..35  31k..35k     40
3     systems  junior  21..25  46k..50k     20
4     systems  senior  31..35  66k..70k      5
5     systems  junior  26..30  46k..50k      3
6     systems  senior  41..45  66k..70k      3
7   marketing  senior  36..40  46k..50k     10
8   marketing  junior  31..35  41k..45k      4
9   secretary  senior  46..50  36k..40k      4
10  secretary  junior  26..30  26k..30k      6>

In [3]:
#https://medium.com/@rakendd/decision-tree-from-scratch-9e23bcfb4928
def entropy(df,target_attribute):
    entropy_node = 0  #Initialize Entropy
    values = df[target_attribute].unique() 
    for value in values:
        fraction = df.groupby(['status'])['count'].agg('sum')[value]/sum(df['count'])
        entropy_node += -fraction*np.log2(fraction)
    print("entropy: ",round(entropy_node,4))
    return round(entropy_node,4)

def info_gain(df,attribute,target_attribute):
    target_variables = df[target_attribute].unique() 
    variables = df[attribute].unique()    
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            attribute_num = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
            num = attribute_num['count'][attribute_num[attribute]==variable][attribute_num[target_attribute]==target_variable].sum()
            attribute_den = df.groupby([attribute],as_index=False)['count'].agg('sum')
            den = attribute_den['count'][attribute_den[attribute]==variable].sum()
            fraction = num/(den+eps)  
            entropy_each_feature += fraction*log(fraction+eps) 
        fraction2 = den/df['count'].sum()
        entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste
    print("entropy_",attribute,":",round(entropy_attribute,4))
    return round(entropy(df,'status') - entropy_attribute,4)

print("Info_gain = ","using department :",info_gain(df,'department','status'))
print("Info_gain = ","using age :",info_gain(df,'age','status'))
print("Info_gain = ","using salary :",info_gain(df,'salary','status'))

entropy_ department : 0.8504
entropy:  0.899
Info_gain =  using department : 0.0486
entropy_ age : 0.4743
entropy:  0.899
Info_gain =  using age : 0.4247
entropy_ salary : 0.3615
entropy:  0.899
Info_gain =  using salary : 0.5375


In [146]:
def split_info(df,attribute):
    values = df[attribute].unique() 
    split_info = 0
    for value in values:
        fraction = df.groupby([attribute])['count'].agg('sum')[value]/sum(df['count'])
        split_info += -fraction*np.log2(fraction)
    return round(split_info,4)
print("Gain_ratio_","department",round(info_gain(df,'department','status')/split_info(df,'department'),4))
print("Gain_ratio_","age",round(info_gain(df,'age','status')/split_info(df,'age'),4))
print("Gain_ratio_","salary",round(info_gain(df,'salary','status')/split_info(df,'salary'),4))

entropy_ department : 0.8504
entropy:  0.899
Gain_ratio_ department 0.035
entropy_ age : 0.4743
entropy:  0.899
Gain_ratio_ age 0.2261
entropy_ salary : 0.3615
entropy:  0.899
Gain_ratio_ salary 0.2672


In [130]:
def get_list(df,attribute):
  items = df[attribute].unique() 
  #print(items)
  from itertools import combinations
  tmp = []
  for j in range(1,len(items)):
    for i in combinations(items, j):
      tmp.append(i)
  return list(set(tmp))
get_list(df,'department')


["('sales', 'marketing', 'secretary'),('sales', 'secretary')", "('sales', 'marketing', 'secretary'),('marketing', 'secretary')", "('sales', 'marketing', 'secretary'),('sales', 'systems')", "('sales', 'marketing', 'secretary'),('secretary',)", "('sales', 'marketing', 'secretary'),('sales', 'marketing')", "('sales', 'marketing', 'secretary'),('systems', 'secretary')", "('sales', 'marketing', 'secretary'),('systems', 'marketing')", "('sales', 'marketing', 'secretary'),('sales', 'systems', 'marketing')", "('sales', 'marketing', 'secretary'),('systems',)", "('sales', 'marketing', 'secretary'),('systems', 'marketing', 'secretary')", "('sales', 'marketing', 'secretary'),('marketing',)", "('sales', 'marketing', 'secretary'),('sales', 'systems', 'secretary')", "('sales', 'marketing', 'secretary'),('sales',)", "('sales', 'secretary'),('marketing', 'secretary')", "('sales', 'secretary'),('sales', 'systems')", "('sales', 'secretary'),('secretary',)", "('sales', 'secretary'),('sales', 'marketing')"

In [144]:
def gini(df,attribute,target_attribute):
    target_variables = df[target_attribute].unique() 
    all_variables = df[attribute].unique()  
    entropy_attribute = 0
    variables = get_list(df,attribute)
    for variable in variables:
        variable = list(variable)
        entropy_each_feature = 0
        attribute_num = df.groupby([attribute],as_index=False)['count'].agg('sum')
        num1 = 0
        for i in variable:
            num1 += attribute_num['count'][attribute_num[attribute] == i].sum()
        #print(num1)
        #for target_variable in target_variables:
        attribute_num2 = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
        num2 = 0
        for i in variable:
          num2 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='junior'].sum()
        num3 = 0
        for i in variable:
          num3 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='senior'].sum()
        #print(num1,num2,num3)
        #print(variable)
        remaining_variables = li_dif = [i for i in all_variables if i not in variable] 
        num4 = 0
        for i in remaining_variables:
            num4 += attribute_num['count'][attribute_num[attribute] == i].sum()
        #print(num1)
        #for target_variable in target_variables:
        attribute_num2 = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
        num5 = 0
        for i in remaining_variables:
          num5 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='junior'].sum()
        num6 = 0
        for i in remaining_variables:
          num6 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='senior'].sum()
        #print(num4,num5,num6)
        #print(remaining_variables)
        gini_index = (num1*(1 - (num2/num1)**2 - (num3/num1)**2) +  num4*(1 - (num5/num4)**2 - (num6/num4)**2))/sum(df['count'])
        print("gini index of",variable,round(gini_index,4))
    #return round(entropy(df,'status') - entropy_attribute,4)
gini(df,'department','status')
gini(df,'age','status')
gini(df,'salary','status')  

gini index of ['sales', 'marketing', 'secretary'] 0.4302
gini index of ['sales', 'secretary'] 0.4263
gini index of ['marketing', 'secretary'] 0.4072
gini index of ['sales', 'systems'] 0.4072
gini index of ['secretary'] 0.4307
gini index of ['sales', 'marketing'] 0.4313
gini index of ['systems', 'secretary'] 0.4313
gini index of ['systems', 'marketing'] 0.4263
gini index of ['sales', 'systems', 'marketing'] 0.4307
gini index of ['systems'] 0.4302
gini index of ['systems', 'marketing', 'secretary'] 0.4245
gini index of ['marketing'] 0.4021
gini index of ['sales', 'systems', 'secretary'] 0.4021
gini index of ['sales'] 0.4245
gini index of ['26..30', '46..50'] 0.3773
gini index of ['31..35', '46..50'] 0.3832
gini index of ['36..40', '46..50'] 0.3447
gini index of ['21..25', '36..40', '46..50'] 0.4268
gini index of ['41..45'] 0.4143
gini index of ['31..35', '21..25'] 0.4272
gini index of ['26..30', '21..25'] 0.2889
gini index of ['31..35', '26..30', '46..50'] 0.4286
gini index of ['31..35',